In [1]:
# Cell 1 — robust fetch of the 360+ team slugs

# ── 1) install & import cache tools ──────────────────────────────────────
!pip install requests-cache   # run once; comment out after it’s installed

import requests_cache, requests, certifi, re, time
from bs4 import BeautifulSoup

# set up a cached session (caches to 'teamlist_cache.sqlite')
requests_cache.install_cache("teamlist_cache", expire_after=86400)
session = requests_cache.CachedSession()

SEASON   = 2025
BASE_URL = f"https://www.sports-reference.com/cbb/seasons/{SEASON}-school-stats.html"
HEADERS  = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"}

# ── 2) fetch once, with retry/back-off ───────────────────────────────────
def fetch_teamlist(url, max_retries=3):
    backoff = 1
    for attempt in range(1, max_retries+1):
        resp = session.get(url, headers=HEADERS,
                           verify=certifi.where(), timeout=10)
        if resp.status_code == 200:
            return resp.text
        if resp.from_cache:
            # if we’re reading from cache but it’s stale, just use it
            return resp.text
        if resp.status_code == 429:
            print(f"⚠️  429 rate limit, sleeping {backoff}s (attempt {attempt})")
            time.sleep(backoff)
            backoff *= 2
        else:
            resp.raise_for_status()
    raise RuntimeError(f"Failed to fetch {url} after {max_retries} retries")

html = fetch_teamlist(BASE_URL)

# ── 3) parse out the team slugs ────────────────────────────────────────
soup  = BeautifulSoup(html, "lxml")
links = soup.select("table#basic_school_stats a[href*='/cbb/schools/']")
teams = {
    a.text.strip(): re.search(r"/cbb/schools/([^/]+)/", a["href"]).group(1)
    for a in links
}

print(f"✅  Teams found: {len(teams)}")   # should be ~360+




[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
✅  Teams found: 364


In [2]:
# Cell 2 — fetch_schedule() reads each team’s table and extracts site via data-stat

import pandas as pd
from io import StringIO
import certifi
from bs4 import BeautifulSoup

def fetch_schedule(slug, team_name):
    """
    Return DataFrame with columns:
      date, opponent, team_pts, opp_pts, site, team
    """
    url  = f"https://www.sports-reference.com/cbb/schools/{slug}/{SEASON}-schedule.html"
    resp = session.get(url, headers=HEADERS, verify=certifi.where(), timeout=10)
    resp.raise_for_status()

    # 1) Load into pandas to get core table
    df = pd.read_html(StringIO(resp.text), match="Date", flavor="lxml")[0]
    if "G" in df.columns:
        df = df[df["G"] != "G"]

    # 2) Scrape the raw <td data-stat="game_location"> cells
    soup  = BeautifulSoup(resp.text, "lxml")
    table = soup.find("table", id="schedule")
    sites_raw = [td.get_text(strip=True) for td in table.select("td[data-stat='game_location']")]
    site = pd.Series(sites_raw, name="site").map({"@":"away","N":"neutral","": "home"}).fillna("home")

    # 3) Clean opponent names (strip leading "@"/"N")
    df["opponent"] = df["Opponent"].astype(str).str.lstrip("@N ").str.strip()

    # 4) Rename date & score columns
    col_map = {"Date":"date", "Tm":"team_pts", "Opp":"opp_pts", "Opp.1":"opp_pts"}
    df = df.rename(columns=col_map)

    # 5) Assemble final DataFrame
    out = pd.DataFrame({
        "date":     df["date"],
        "opponent": df["opponent"],
        "team_pts": df["team_pts"],
        "opp_pts":  df["opp_pts"],
        "site":     site.values,
        "team":     team_name
    })
    return out


In [3]:
# Cell 3 — very-safe full-season scrape in batches to avoid rate-limits

import os
import time
import random
import pandas as pd
from tqdm import tqdm

OUT = f"ncaa_games_{SEASON}.csv"

if os.path.exists(OUT):
    # If the full CSV already exists, just load it
    games = pd.read_csv(OUT, parse_dates=["date"])
    print(f"📥 Loaded existing {OUT} ({len(games)} games)")
else:
    all_games = []
    teams_list = list(teams.items())
    batch_size = 20       # number of teams per batch
    post_batch_sleep = 300  # 5 minutes between batches

    # Loop in batches
    for batch_start in range(0, len(teams_list), batch_size):
        batch = teams_list[batch_start:batch_start + batch_size]
        for team, slug in batch:
            try:
                sched = fetch_schedule(slug, team)  # from Cell 2; uses cached session
                all_games.append(sched)
            except Exception as e:
                print(f"⚠️ Skipping {team}: {e}")
            # polite pause 15–30s between individual requests
            delay = random.uniform(15, 30)
            print(f"⏱ Sleeping {delay:.0f}s after {team}")
            time.sleep(delay)

        # after each batch, rest for 5 minutes (unless it’s the last batch)
        if batch_start + batch_size < len(teams_list):
            print(f"✅ Batch {batch_start//batch_size + 1} complete; sleeping {post_batch_sleep//60}min")
            time.sleep(post_batch_sleep)

    # Combine all team DataFrames
    combined = pd.concat(all_games, ignore_index=True)

    # De-duplicate so each head-to-head game appears once
    combined["key"] = (
        combined["date"].astype(str) + "_" +
        combined[["team","opponent"]]
                .astype(str)
                .apply(lambda x: "_".join(sorted(x)), axis=1)
    )
    games = (
        combined
        .sort_values(["date","team"])
        .drop_duplicates("key")
        .drop(columns="key")
        .reset_index(drop=True)
    )

    # Compute margin and save
    games["margin"] = games["team_pts"] - games["opp_pts"]
    games.to_csv(OUT, index=False)
    print(f"✅ Scraped & saved {len(games)} games → {OUT}")

# Sanity check
display(games.head())
print("Site counts:\n", games.site.value_counts())



⏱ Sleeping 19s after Abilene Christian
⏱ Sleeping 16s after Air Force
⏱ Sleeping 18s after Akron
⏱ Sleeping 22s after Alabama
⏱ Sleeping 21s after Alabama A&M
⏱ Sleeping 22s after Alabama State
⏱ Sleeping 20s after Albany (NY)
⏱ Sleeping 29s after Alcorn State
⏱ Sleeping 21s after American
⏱ Sleeping 27s after Appalachian State
⏱ Sleeping 23s after Arizona
⏱ Sleeping 28s after Arizona State
⏱ Sleeping 29s after Arkansas
⏱ Sleeping 25s after Arkansas State
⏱ Sleeping 20s after Arkansas-Pine Bluff
⏱ Sleeping 17s after Army
⏱ Sleeping 29s after Auburn
⏱ Sleeping 28s after Austin Peay
⏱ Sleeping 26s after Ball State
⏱ Sleeping 17s after Baylor
✅ Batch 1 complete; sleeping 5min
⏱ Sleeping 25s after Bellarmine
⏱ Sleeping 23s after Belmont
⏱ Sleeping 23s after Bethune-Cookman
⏱ Sleeping 22s after Binghamton
⏱ Sleeping 16s after Boise State
⏱ Sleeping 18s after Boston College
⏱ Sleeping 20s after Boston University
⏱ Sleeping 18s after Bowling Green State
⏱ Sleeping 27s after Bradley
⏱ Sleeping

KeyboardInterrupt: 

In [3]:
# Cell 3 — collect every team’s DataFrame into a list

all_games = []
for team, slug in teams.items():
    sched = fetch_schedule(slug, team)   # uses your Cell 2 function
    all_games.append(sched)

# now all_games has 364 DataFrames


HTTPError: 429 Client Error: Too Many Requests for url: https://www.sports-reference.com/cbb/schools/bethune-cookman/2025-schedule.html

In [4]:
# Quick check: is 'teams' in your namespace?
print("teams defined?", 'teams' in globals())
if 'teams' in globals():
    print("Number of teams:", len(teams))
    print("Sample keys:", list(teams.keys())[:5])



teams defined? True
Number of teams: 364
Sample keys: ['Abilene Christian', 'Air Force', 'Akron', 'Alabama', 'Alabama A&M']
